In [3]:
from Neuro_Plotting.plot import plot_surf_parc
import numpy as np
import os
import matplotlib.pyplot as plt
from nilearn.surface import load_surf_data

In [ ]:
mw_loc = '../raw/fs_LR_32k_label/medialwall.annot'
mw_mask = ~load_surf_data(mw_loc).astype('bool')

def clean_name(parc):
    base = parc.split('/')[-1]
    name = base.replace('_', ' ').replace('.npy', '')
    name = name.replace('-', ' ')
    return name

parcs = os.listdir('../parcels/')
parcs = [os.path.join('../parcels/', p) for p in parcs]

os.makedirs('Figures/parcels/', exist_ok=True)

for parc in parcs:
    
    if '_prob' in parc:
        continue
        
    save_loc = 'Figures/parcels/' + parc.split('/')[-1].replace('.npy', '.png')
    if os.path.exists(save_loc):
        continue
        
    parcel = np.load(parc)
    parcel[mw_mask] = 0
        
    plot_surf_parc(parcel, title=clean_name(parc))
    plt.savefig(save_loc, dpi=100, transparent=True, bbox_inches='tight')
    plt.close()